In [1]:
import pandas as pd, numpy as np, os, sys, glob, nibabel as nib
from scipy import ndimage


In [2]:
csv_path = './20211104_label_1-350.csv'
table =  pd.read_csv(csv_path)
table_3t = table[table['1/0: 3T/1.5T MRI']==1.0]
table_3t_train = table_3t[table_3t['Valid data']!='V']
table_3t_valid = table_3t[table_3t['Valid data']=='V']
table_3t_test = table_3t_valid[table_3t_valid['排除']=='Test data']
table_3t_valid = table_3t_valid[table_3t_valid['排除']!='Test data']

len(table_3t_test)

45

In [3]:
mask3 = np.concatenate((np.zeros((384,384,10)), np.zeros((384,384,15))), axis=-1)
print(mask3.shape)

(384, 384, 25)


In [4]:
class nii_process:
    def __init__(self, base_):
        self.base_ = base_
        self.volume = np.array([])

    def normalize(self, volume):
        img_o = np.float32(volume.copy())
        m = np.mean(img_o)
        s = np.std(img_o)
        volume = np.divide((img_o - m), s)
        image = volume.astype("float32")
        return image

    def process_scan(self, dstype,path_img, path_msg):

        image_o = nib.load(path_img)
        masks_o = nib.load(path_msg)
        affine = image_o.header.get_best_affine()

        if len(image_o.shape) == 4:
            image = image_o.get_fdata()
            masks = masks_o.get_fdata()
            width,height,queue,_ = image.shape
            image = image[:,:,:,1]
            image = np.reshape(image,(width,height,queue))
            masks = np.reshape(masks,(width,height,queue))
        else:
            image = image_o.get_fdata()
            masks = masks_o.get_fdata()

        image = self.normalize(image)
        if dstype!='test':
            image = np.where(masks, image, image*0)

        self.slice_n = image.shape[-1]
        nii_name_slices = (os.path.split(path_img))
        nii_name_slices = nii_name_slices[1].split('.')[0]
        # print(nii_name_slices)
        if image.shape[-1] !=26:
            black_slice = np.zeros((384,384,))
            new_image = np.concatenate((image, np.zeros((384,384, (26-image.shape[-1])))), axis=-1)
            new_mask = np.concatenate((masks, np.zeros((384,384, (26-masks.shape[-1])))), axis=-1)
            adjusted_seg = nib.Nifti1Image(new_image, affine)
            adjusted_msk = nib.Nifti1Image(new_mask, affine)
            adjusted_seg.header['pixdim'] = image_o.header['pixdim']
            adjusted_msk.header['pixdim'] = masks_o.header['pixdim']
            # Save as NiBabel file
            adjusted_seg.to_filename(os.path.join(self.base_, dstype,f'{nii_name_slices[0:-1]}.nii.gz'))
            adjusted_msk.to_filename(os.path.join(self.base_, 'test_mask',f'{nii_name_slices[0:-1]}.nii.gz'))

In [5]:
# process_stack = ['train','valid','test']
process_stack = ['test']
prepare_data = nii_process('./dataset/S2_data/')
for i in process_stack:
    _list = sorted(os.listdir(os.path.join('./dataset/original_data/', i)))
    _len = len(_list)
    for j in range(0, _len, 2):
        prepare_data.process_scan(i, f'./dataset/original_data/{i}/{_list[j]}', f'./dataset/original_data/{i}/{_list[j+1]}')